### *Module Loading*

In [ ]:
import sys
from IPython.display import display as ip_display

### *External Module Loading*

In [ ]:
external_modules_path = '..\\nn_likelihood_modules'
sys.path.append(external_modules_path)

In [ ]:
from basic_network_structure import *
from common_imports import *
from common_use_functions import *
from constant import *
from defined_data_structure import *
from defined_network_structure import *
from distribution_calculation import *
from direct_inference_likelihood import *
from experim_neural_network import *
from experim_preparation import *
from generate_activation_level import *
from pytorch_model_predict import *
from vector_preprocessing import *
from ResNet import *
from experim_ResNet import *
from cifar_10_data_prep import *
from pytorch_swintransformer_modified import *
from tiny_imagenet_data_prep import *
from sensitivity_analysis import *

### *GPU verification*

In [ ]:
# Get the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nb_gpu = torch.cuda.device_count()
if nb_gpu > 0:
    print(torch.cuda.get_device_name(0))
else:
    print("CPU")

### *Working directory*

In [ ]:
# Current path
current_path = os.path.abspath(os.getcwd())

### *Load configurations and data*

In [ ]:
"""
All the parameters in this part should be configured
"""
# Experience path
experim_path = current_path

# File extensions
json_ext = '.json'
np_ext = '.npy'
csv_ext = '.csv'

# ResNet model prefix
model_name_prefix = 'swin'

# Image max pixel value
image_max_pix_val = 255

# Tested sets name
train_set_name = 'train'
test_set_name = 'test'
valid_set_name = 'valid'
input_extension = 'X'
label_extension = 'Y'

# Save paths
model_save_path = path_join(experim_path, 'experim_models_swin')

"""
The following parameters should be configured according to your experiments
"""

# Trained model name 
trained_net_name = 'swin_b_v2_256' # You can select any model from the "experim_models_swin" folder 

# Network related params
net_model_name = 'swin_b_v2_256'# The model name should be coherent with your chosen model

# Tiny-imagenet datafolder path
tiny_imagenet_path = 'D:\\Doctorat\\research\\Tiny_imagenet_experim\\tiny-imagenet-200-reform\\'
tiny_imagenet_val_path = path_join(tiny_imagenet_path, 'reform_val')
tiny_imagenet_train_path = path_join(tiny_imagenet_path, 'reform_train')

# Tiny-imagenet-c datafolder path
tiny_imagenet_c_path = 'D:\\Doctorat\\research\\Tiny_imagenet_experim\\Tiny-ImageNet-C-reform\\'

# Dataset general informations
data_set_infos = {
    'nb_classes' : 200
}

# Distance decision filtering threshold
std_threshold_coeff = 2

# The method to determining the significant neurons (mean or most_common)
sobol_filter_method = 'mean'

# Rscript launch params
Rscript_path = 'C:\\Program Files\\R\\R-4.3.3\\bin\\Rscript.exe'

"""
"""

# The output folder
output_path = path_join(experim_path, 'output')

# Distribution file
train_distribution_filename = path_join(output_path, 'distribution_' + train_set_name + '_set.csv')

# Train distance information registration path
train_dists_info_filename = 'train_dist_infos'
train_dists_info_path = path_join(output_path, train_dists_info_filename + json_ext)

# Build the class list
class_list = list(range(data_set_infos['nb_classes']))

# Batch size for the dataloader creation
torch_batch_size = 4

In [ ]:
# Create the folders
create_directory(output_path)

### *Experiment preparation*

In [ ]:
# Get the tiny imagenet training set and test set
tiny_imagenet_train_dataset, tiny_imagenet_test_dataset = get_tiny_imagenet_without_transform(tiny_imagenet_train_path, tiny_imagenet_val_path)

In [ ]:
# The column names of the filtering results
column_names_OOD_filtering = ['transformation', 'nb_examples', 'nb_OOD', 'nb_InD', 'total_acc', 'OOD_acc', 'InD_acc']

### *Load the trained Network*

In [ ]:
# Load the network
trained_net = load_model_by_net_name(model_save_path, trained_net_name)

### *Tiny imagenet dataset preparation*

In [ ]:
# Dataloader building
train_loader = create_loader_from_torch_dataset(tiny_imagenet_train_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)
test_loader = create_loader_from_torch_dataset(tiny_imagenet_test_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)

### *Distribution and Training set likelihood distance information generation*

In [ ]:
# Move to gpu
trained_net.cuda()

In [ ]:
# Get the activation levels
train_actLevels = obtain_activation_levels(trained_net,
                                           train_loader, train_set_name, with_predict_class=True, loss_type='cross_entropy')

In [ ]:
# Get the activation levels only on the correctly predicted examples
correctly_predicted_bools = (train_actLevels['class'] == train_actLevels['predict_class']).reshape(-1)
correctly_predicted_train_actLevels = copy.deepcopy(train_actLevels)
for info in correctly_predicted_train_actLevels:
    if info != 'actLevel':
        correctly_predicted_train_actLevels[info] = correctly_predicted_train_actLevels[info][correctly_predicted_bools]
    else:
        for layerId in correctly_predicted_train_actLevels[info]:
            correctly_predicted_train_actLevels[info][layerId] = correctly_predicted_train_actLevels[info][layerId][correctly_predicted_bools]

In [ ]:
## Distribution generation
# Generate the distribution
train_distributions = generate_distributions(correctly_predicted_train_actLevels, train_set_name, trained_net_name)
# Save the distribution
train_distribution_filename = save_distributions(output_path, train_distributions, train_set_name)

In [ ]:
# Generate the likelihood information per class on the training set
# Last hidden layer index and activation levels
last_hidden_layerId = list(correctly_predicted_train_actLevels['actLevel'].keys())[-1]
# Calculate the likelihood on the last hidden layer
train_distrib_distances, train_distrib_map_dict = layer_ref_likelihood_experim(train_actLevels, last_hidden_layerId,
                               train_distribution_filename, predicted_class=False, use_absolute_module_path=True)

In [ ]:
# Take the correctly predicted ones
correct_train_last_hidden_distances = train_distrib_distances[train_distrib_distances['classId'] == train_distrib_distances['predicted_classId']].copy(deep=True)

In [ ]:
## Calculate the information and save it
# Information calculation
class_dist_infos = build_layer_train_set_infos_ref_ver(correct_train_last_hidden_distances, class_list)
# Register the calculated information
store_dict_as_json(train_dists_info_path, class_dist_infos)

### *Evaluate the likelihood on the whole training set*

In [ ]:
# Calculate the likelihood on the last hidden layer
train_distances, train_map_dict = layer_ref_likelihood_experim(train_actLevels, last_hidden_layerId,
                               train_distribution_filename, predicted_class=True, use_absolute_module_path=True)

In [ ]:
# Normalize the whole distances
train_distances = normalize_distances_ref_ver(train_distances, train_map_dict, class_dist_infos)

In [ ]:
# Find the decisions to be filtered
train_filtered_index = filter_decision_based_on_train_infos_norm_and_ref_ver(train_distances, class_dist_infos,
                                                                             std_threshold_coeff, original_class=False)

In [ ]:
# Visualize the filtered training set examples
if len(train_filtered_index) != 0:
    ip_display(train_distances.loc[train_filtered_index,:])
else:
    print('All the example images are identified as good inputs by the likelihood distance.')

### *Evaluate the likelihood on the test set*

In [ ]:
# Get the activation levels
test_actLevels = obtain_activation_levels(trained_net,
                                           test_loader, 'test', with_predict_class=True, loss_type='cross_entropy')

In [ ]:
# Calculate the likelihood on this layer
test_distances, test_map_dict = layer_ref_likelihood_experim(test_actLevels, last_hidden_layerId,
                               train_distribution_filename, predicted_class=True, use_absolute_module_path=True)

In [ ]:
# Normalize the distances
test_distances = normalize_distances_ref_ver(test_distances, test_map_dict, class_dist_infos)

In [ ]:
# Find the decisions to be filtered
test_filtered_index = filter_decision_based_on_train_infos_norm_and_ref_ver(test_distances, class_dist_infos,
                                                                             std_threshold_coeff, original_class=False)

In [ ]:
# Visualize the filtered test examples
if len(test_filtered_index) != 0:
    ip_display(test_distances.loc[test_filtered_index,:])
else:
    print('All the example images are identified as good inputs by the likelihood distance.')

### *Sobol index evaluation*

In [ ]:
# Get the corresponding data
last_hidden_actLevels = train_actLevels['actLevel'][last_hidden_layerId][correctly_predicted_bools.reshape(-1)]

In [ ]:
# Get the last layer parameters
model_params = get_model_parameters(trained_net, to_numpy=True)
final_linear_params = model_params['head_out.0']

In [ ]:
# Normalization of weight and input
# Copy the original activation levels and weights
normalized_last_hidden_actLevel = copy.deepcopy(last_hidden_actLevels)
upped_final_linear_params = copy.deepcopy(final_linear_params)
# Check the min and max values of each neuron
last_hidden_actLevel_max = np.max(last_hidden_actLevels, axis=0)
last_hidden_actLevel_min = np.min(last_hidden_actLevels, axis=0) # Not used, just for verification
# Iterate over the maximum values and normalize the input
for index, neuron_max in enumerate(last_hidden_actLevel_max):
    if neuron_max != 0:
        normalized_last_hidden_actLevel[:,index] = normalized_last_hidden_actLevel[:,index] / neuron_max
        upped_final_linear_params['weight'][:,index] = upped_final_linear_params['weight'][:,index] * neuron_max

In [ ]:
# Build the final linear parameters per class and assign the real data
final_linear_param_per_class = build_per_class_linear_model(upped_final_linear_params)
data = normalized_last_hidden_actLevel
# Number of variables
nb_vars = data.shape[1]

In [ ]:
## Build the X and y for the sobol index evaluation in R
# Get the neuron names
neuron_names = ['neuron_'+str(index) for index in range(last_hidden_actLevels.shape[1])]
# Build the X dataframe
R_X = pd.DataFrame(data, columns=neuron_names)

In [ ]:
# Build the dataframe that stores weights and bias
R_network_params_data = []
R_network_params_columns = ['classId', *(['weight_'+str(index) for index in range(len(final_linear_param_per_class[0]['weight']))]), 'bias']
for classId in final_linear_param_per_class:
    R_network_params_data.append([classId, *list(final_linear_param_per_class[classId]['weight']), final_linear_param_per_class[classId]['bias']])
R_network_params = pd.DataFrame(R_network_params_data, columns=R_network_params_columns)
save_df_to_csv(path_join(output_path, 'R_network_params.csv'),R_network_params)

In [ ]:
# Sample size
N = 10000

In [ ]:
# Generate two random samples from R_X
A_index = generate_sample_index(data, N, replace=False)
B_index = generate_sample_index_exclude_items(data, N, A_index, replace=False)
R_X_A = R_X.loc[A_index,:].copy(deep=True).reset_index(drop=True)
R_X_B = R_X.loc[B_index,:].copy(deep=True).reset_index(drop=True)
# Save the random samples from R_X
save_df_to_csv(path_join(output_path, 'R_X_A.csv'),R_X_A)
save_df_to_csv(path_join(output_path, 'R_X_B.csv'),R_X_B)

In [ ]:
# You could choose sobolrank (only first order indices), sobolEff (first and total indices) or shapleysobol_knn (first and total indices)
R_sobol_method = 'sobolEff'
R_sobol_script = path_join(experim_path, R_sobol_method+'_eval.R')

In [ ]:
# Execute the R script (The path to read the data is given as arguments (i.e., experim_path+'\\'))
ouput_R = run([Rscript_path, '--vanilla', R_sobol_script, output_path+'\\', str(data_set_infos['nb_classes'])], shell=True) 

In [ ]:
## Get the important variables per class
# Initialize the determined variables as None
important_variables_per_class = None
if R_sobol_method == 'sobolEff':
    # Load the sobol indices (first and total) and convert it to a dictionary
    R_first_order_sobol_per_class_dict = {}
    R_total_order_sobol_per_class_dict = {}
    for classId in class_list:
        R_current_class_first_order_sobol_indices = read_csv_to_pd_df(path_join(output_path, R_sobol_method+'_fs_'+str(classId)+csv_ext))
        R_current_class_total_order_sobol_indices = read_csv_to_pd_df(path_join(output_path, R_sobol_method+'_tt_'+str(classId)+csv_ext))
        R_first_order_sobol_per_class_dict[classId] = R_current_class_first_order_sobol_indices['S.original'].to_dict()
        R_total_order_sobol_per_class_dict[classId] = R_current_class_total_order_sobol_indices['S.original'].to_dict()
    # Get the important variables per class
    important_variables_per_class = important_variables_R_first_and_total_order_analysis(R_first_order_sobol_per_class_dict, 
                                                                                         R_total_order_sobol_per_class_dict,
                                                                                         class_list, filter_method=sobol_filter_method)

In [ ]:
# Build the dataframe that contains the number of neurons
determined_nb_important_vars_per_class = []
for classId in important_variables_per_class:
    class_nb_important_vars = len(list(important_variables_per_class[classId].keys()))
    determined_nb_important_vars_per_class.append([classId, nb_vars, class_nb_important_vars])
determined_nb_important_vars_per_class_df = pd.DataFrame(determined_nb_important_vars_per_class, columns=['classId', 'nb_neurons', 'nb_important_neurons'])

In [ ]:
# Save the important variables
store_dict_as_json(path_join(output_path, trained_net_name+'_important_neurons.json'), important_variables_per_class)
save_df_to_csv(path_join(output_path, trained_net_name+'_nb_important_neurons.csv'), determined_nb_important_vars_per_class_df)

### *Likelihood calculation based only on the important variables*

In [ ]:
# Build the sorted total important variable(neuron) indices
sorted_important_var_by_class = {}
for classId in important_variables_per_class:
    sorted_important_var_by_class[classId] = sorted(list(important_variables_per_class[classId]))
# Build the mapping dictionary to modify neuron indices
important_var_map_dict_by_class = {}
for classId in sorted_important_var_by_class:
    important_var_map_dict_by_class[classId] = build_map_to_index_dict(sorted_important_var_by_class[classId])

In [ ]:
## Take the training set distribution only for the important neurons
# Read the training set distribution
train_whole_distribution = pd.read_csv(train_distribution_filename, sep=' ')
# Take the distribution for different classes
train_important_var_distrib_by_class = {}
for classId in sorted_important_var_by_class:
    # Get the important variables of the current class
    current_important_neuron_indices = sorted_important_var_by_class[classId]
    # Take only the last hidden layer distribution (We calculate the likelihood only based on this)
    train_whole_last_hidden_distribution = train_whole_distribution[train_whole_distribution['layerId'] == last_hidden_layerId].copy(deep=True)
    # Filter the distribution on the last hidden layer
    important_var_train_whole_last_hidden_distribution = train_whole_last_hidden_distribution[train_whole_last_hidden_distribution['nodeId'].isin(current_important_neuron_indices)]
    # Map the node Id
    train_important_var_distrib_by_class[classId] = important_var_train_whole_last_hidden_distribution.replace({"nodeId": important_var_map_dict_by_class[classId]}).reset_index(drop=True)

In [ ]:
# Save the temporarily generated distribution files by class
for classId in train_important_var_distrib_by_class:
    save_df_to_csv(path_join(output_path, 'distribution_important_var_class_'+str(classId)+csv_ext), train_important_var_distrib_by_class[classId], sep=' ')

In [ ]:
# Calculate the likelihood based on only the important variables according to the original class
train_distrib_likelihood_important_vars, train_distrib_map_dict_important_vars = calculate_likelihood_with_important_vars_saving_memory_ref_ver(output_path, train_actLevels,
                                                                                               sorted_important_var_by_class,
                                                                                               last_hidden_layerId, predicted_class=False,
                                                                                               use_absolute_module_path=True)

In [ ]:
# Get only the correctly predicted examples
correct_train_likelihood_important_vars = train_distrib_likelihood_important_vars[train_distrib_likelihood_important_vars['classId'] == train_distrib_likelihood_important_vars['predicted_classId']]
# Calculate the distance (with only important variables) information
class_dist_infos_important_vars = build_layer_train_set_infos_ref_ver(correct_train_likelihood_important_vars, class_list)
# Register the calculated information
store_dict_as_json(path_join(output_path, 'train_dist_infos_important_var'+json_ext), class_dist_infos_important_vars)

In [ ]:
# Calculate the likelihood based on only the important variables
train_likelihood_important_vars, train_map_dict_important_vars = calculate_likelihood_with_important_vars_saving_memory_ref_ver(output_path, train_actLevels,
                                                                                               sorted_important_var_by_class,
                                                                                               last_hidden_layerId, predicted_class=True,
                                                                                               use_absolute_module_path=True)
test_likelihood_important_vars, test_map_dict_important_vars = calculate_likelihood_with_important_vars_saving_memory_ref_ver(output_path, test_actLevels,
                                                                                              sorted_important_var_by_class,
                                                                                              last_hidden_layerId, predicted_class=True,
                                                                                              use_absolute_module_path=True)

In [ ]:
# Normalize the distances
train_likelihood_important_vars = normalize_distances_ref_ver(train_likelihood_important_vars, 
                                                              train_map_dict_important_vars, class_dist_infos_important_vars)
test_likelihood_important_vars = normalize_distances_ref_ver(test_likelihood_important_vars,
                                                             test_map_dict_important_vars, class_dist_infos_important_vars)

In [ ]:
# Find the decisions to be filtered
train_filtered_index_important_vars = filter_decision_based_on_train_infos_norm_and_ref_ver(train_likelihood_important_vars,
                                                                                           class_dist_infos_important_vars,
                                                                                           std_threshold_coeff, original_class=False)
test_filtered_index_important_vars = filter_decision_based_on_train_infos_norm_and_ref_ver(test_likelihood_important_vars,
                                                                                          class_dist_infos_important_vars,
                                                                                          std_threshold_coeff, original_class=False)

In [ ]:
# Original Tiny-imagenet dataset filtering results (with only the signifiant neurons)
# Accuracy evaluation on the filtering
built_tiny_imagenet_original_important_vars_filtering_result = []
print('Evaluation with only the important neurons:')
print('Training set:')
built_tiny_imagenet_original_important_vars_filtering_result.append(['train',
                                                *evaluate_filtering(train_likelihood_important_vars, train_filtered_index_important_vars, 'train')])
print()
print('Test set:')
built_tiny_imagenet_original_important_vars_filtering_result.append(['test',
                                                *evaluate_filtering(test_likelihood_important_vars, test_filtered_index_important_vars, 'test')])
# Build the result dataframe
tiny_imagenet_original_important_vars_OOD_df = pd.DataFrame(built_tiny_imagenet_original_important_vars_filtering_result, columns=column_names_OOD_filtering)
# Save the result dataframe
save_df_to_csv(path_join(output_path, trained_net_name+'_tiny_imagenet_important_vars_OOD_result.csv'), tiny_imagenet_original_important_vars_OOD_df)

In [ ]:
# Original Tiny-imagenet dataset filtering results (with all neurons)
# Accuracy evaluation on the filtering
built_tiny_imagenet_original_filtering_result = []
print('Evaluation with all the neurons:')
print('Training set:')
built_tiny_imagenet_original_filtering_result.append(['train', *evaluate_filtering(train_distances, train_filtered_index, 'train')])
print()
print('Test set:')
built_tiny_imagenet_original_filtering_result.append(['test', *evaluate_filtering(test_distances, test_filtered_index, 'test')])
# Build the result dataframe
tiny_imagenet_original_OOD_df = pd.DataFrame(built_tiny_imagenet_original_filtering_result, columns=column_names_OOD_filtering)
# Save the result dataframe
save_df_to_csv(path_join(output_path, trained_net_name+'_tiny_imagenet_OOD_result.csv'), tiny_imagenet_original_OOD_df)

In [ ]:
# Save the train and test set whole likelihood distances
save_df_to_csv(path_join(output_path, trained_net_name+'_train_sobol_likelihood.csv'), train_likelihood_important_vars)
save_df_to_csv(path_join(output_path, trained_net_name+'_test_sobol_likelihood.csv'), test_likelihood_important_vars)
save_df_to_csv(path_join(output_path, trained_net_name+'_train_likelihood.csv'), train_distances)
save_df_to_csv(path_join(output_path, trained_net_name+'_test_likelihood.csv'), test_distances)

### *Tiny-imagenet-c experiments*

In [ ]:
## Read the Tiny-imagnet-c dataset
# Get the content in the folder
transformation_types = [transformation_type for transformation_type in contents_of_folder(tiny_imagenet_c_path)]
# Load all the transformation datasets
load_tiny_imagenet_c_datasets = {}
for transformation_type in transformation_types:
    load_tiny_imagenet_c_datasets[transformation_type] = datasets.ImageFolder(
                                                        root=path_join(tiny_imagenet_c_path, transformation_type),
                                                        transform=v2.Compose([v2.ToTensor()]),
                                                    )

In [ ]:
# Show a Tiny-imagenet-c example
plt.imshow(load_tiny_imagenet_c_datasets['brightness_2'][1300][0].permute(1,2,0), interpolation='nearest')
plt.show()
print(load_tiny_imagenet_c_datasets['brightness_2'][1300][1])

In [ ]:
# Build the dataloader and evaluate the activation levels
tiny_imagenet_c_actLevels = {}
for transformation_type in load_tiny_imagenet_c_datasets:
    # Build the loader         
    current_transformed_loader = create_loader_from_torch_dataset(load_tiny_imagenet_c_datasets[transformation_type],
                                                                  batch_size=torch_batch_size, shuffle=False, num_workers=0)
    # Evaluate the activation levels
    tiny_imagenet_c_actLevels[transformation_type] = obtain_activation_levels(trained_net,
                                       current_transformed_loader, transformation_type, with_predict_class=True, loss_type='cross_entropy')

In [ ]:
## Whole likelihood distances experiment
# Calculate the likelihood on the last hidden layer
tiny_imagenet_c_distances = {}
tiny_imagenet_c_map_dicts = {}
for transformation_type in tiny_imagenet_c_actLevels:
    tiny_imagenet_c_distances[transformation_type], tiny_imagenet_c_map_dicts[transformation_type] = layer_ref_likelihood_experim(tiny_imagenet_c_actLevels[transformation_type], last_hidden_layerId,
                               train_distribution_filename, predicted_class=True, use_absolute_module_path=True)

In [ ]:
# Normalize the distances
for transformation_type in tiny_imagenet_c_distances:
    tiny_imagenet_c_distances[transformation_type] = normalize_distances_ref_ver(tiny_imagenet_c_distances[transformation_type],
                                                                                 tiny_imagenet_c_map_dicts[transformation_type],
                                                                                 class_dist_infos)

In [ ]:
# Save the whole likelihood dataframes
for transformation_type in tiny_imagenet_c_distances:
    save_df_to_csv(path_join(output_path, trained_net_name+'_'+transformation_type+'_likelihood.csv'), tiny_imagenet_c_distances[transformation_type])

In [ ]:
# Likelihood filtering
tiny_imagenet_c_filtered_index = {}
for transformation_type in tiny_imagenet_c_distances:
    tiny_imagenet_c_filtered_index[transformation_type] = filter_decision_based_on_train_infos_norm_and_ref_ver(tiny_imagenet_c_distances[transformation_type],
                                                                                                                class_dist_infos,
                                                                                                                std_threshold_coeff,
                                                                                                                original_class=False)

In [ ]:
# Accuracy evaluation on the filtering
built_tiny_imagenet_c_filtering_result = []
for transformation_type in tiny_imagenet_c_filtered_index:
    print(transformation_type, 'set:')
    tiny_imagenet_c_filtering_result = evaluate_filtering(tiny_imagenet_c_distances[transformation_type], tiny_imagenet_c_filtered_index[transformation_type], transformation_type)
    print()
    built_tiny_imagenet_c_filtering_result.append([transformation_type, *tiny_imagenet_c_filtering_result])
# Build the result dataframe
tiny_imagenet_c_OOD_df = pd.DataFrame(built_tiny_imagenet_c_filtering_result, columns=column_names_OOD_filtering)
# Save the result dataframe
save_df_to_csv(path_join(output_path, trained_net_name+'_tiny_imagenet_c_OOD_result.csv'), tiny_imagenet_c_OOD_df)

In [ ]:
# Generate the whole likelihood with the important variables
tiny_imagenet_c_likelihood_important_vars = {}
tiny_imagenet_c_map_dict_important_vars = {}
for transformation_type in tiny_imagenet_c_actLevels:
    tiny_imagenet_c_likelihood_important_vars[transformation_type], tiny_imagenet_c_map_dict_important_vars[transformation_type] = calculate_likelihood_with_important_vars_saving_memory_ref_ver(output_path, 
                                                                                               tiny_imagenet_c_actLevels[transformation_type],
                                                                                               sorted_important_var_by_class,
                                                                                               last_hidden_layerId, predicted_class=True,
                                                                                               use_absolute_module_path=True)

In [ ]:
# Normalize the distances
for transformation_type in tiny_imagenet_c_likelihood_important_vars:
    tiny_imagenet_c_likelihood_important_vars[transformation_type] = normalize_distances_ref_ver(tiny_imagenet_c_likelihood_important_vars[transformation_type], 
                                                              tiny_imagenet_c_map_dict_important_vars[transformation_type], class_dist_infos_important_vars)

In [ ]:
# Save the whole likelihood dataframes
for transformation_type in tiny_imagenet_c_likelihood_important_vars:
    save_df_to_csv(path_join(output_path, trained_net_name+'_'+transformation_type+'_sobol_likelihood.csv'), tiny_imagenet_c_likelihood_important_vars[transformation_type])

In [ ]:
# Likelihood filtering
tiny_imagenet_c_filtered_index_important_vars = {}
for transformation_type in tiny_imagenet_c_likelihood_important_vars:
    tiny_imagenet_c_filtered_index_important_vars[transformation_type] = filter_decision_based_on_train_infos_norm_and_ref_ver(tiny_imagenet_c_likelihood_important_vars[transformation_type],
                                                                                           class_dist_infos_important_vars,
                                                                                           std_threshold_coeff, original_class=False)

In [ ]:
# Accuracy evaluation on the filtering
built_tiny_imagenet_c_important_vars_filtering_result = []
for transformation_type in tiny_imagenet_c_filtered_index_important_vars:
    print(transformation_type, 'set:')
    tiny_imagenet_c_important_vars_filtering_result = evaluate_filtering(tiny_imagenet_c_likelihood_important_vars[transformation_type], tiny_imagenet_c_filtered_index_important_vars[transformation_type], transformation_type)
    print()
    built_tiny_imagenet_c_important_vars_filtering_result.append([transformation_type, *tiny_imagenet_c_important_vars_filtering_result])
# Build the result dataframe
tiny_imagenet_c_important_var_OOD_df = pd.DataFrame(built_tiny_imagenet_c_important_vars_filtering_result, columns=column_names_OOD_filtering)
# Save the result dataframe
save_df_to_csv(path_join(output_path, trained_net_name+'_tiny_imagenet_c_important_vars_OOD_result.csv'), tiny_imagenet_c_important_var_OOD_df)